In [45]:
import pandas as pd

### Cluster의 vrp 정보를 가져온다.

In [46]:
solution = pd.read_csv('solution_cluster_info.csv')

In [47]:
solution

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,50,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,50,1,41.0,23.0,39.0,50.0,NaN,NaN,NaN,NaN,NaN
2,2,50,15,20.0,22.0,6.0,43.0,35.0,50.0,NaN,NaN,NaN
3,3,50,18,40.0,4.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,50,19,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,50,45,49.0,26.0,38.0,0.0,27.0,37.0,8.0,50.0,NaN
6,6,50,31,14.0,7.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
7,7,50,46,17.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,50,32,13.0,9.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
9,9,50,5,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cluster 정보를 나타낸다.

In [48]:
spot = pd.read_csv('clusterinfo.csv')
spot

,Unnamed: 0,lat,lng,cluster
0,1592,35.218610,126.844707,0
1,73,35.219804,126.845734,0
2,1537,35.219165,126.845790,0
3,1705,35.219764,126.846482,0
4,346,35.219443,126.845734,0
...,...,...,...,...
2041,1188,35.220024,126.836280,48
2042,27,35.216719,126.838153,48
2043,466,35.218801,126.838391,48
2044,1786,35.231681,126.851529,49


### Cluster된 정보를 가져오는 함수

In [76]:
def find(cNum):
    cNum = cNum[2:]
    result = pd.DataFrame()
    for i in cNum:
        if(i==50):
            break
        temp = spot[spot['cluster']==i]
        result = pd.concat([result, temp],ignore_index=True,axis=0)
    return result

In [77]:
solutionList = solution.values.tolist()

In [79]:
for i in solutionList:
#     print(i)
#     find(i)
    print(find(i))

Empty DataFrame
Columns: []
Index: []
     Unnamed: 0        lat         lng  cluster
0           462  35.215222  126.850429        1
1           708  35.215360  126.850206        1
2          1559  35.215222  126.850429        1
3           500  35.215222  126.850429        1
4          1572  35.215360  126.849706        1
..          ...        ...         ...      ...
213        1837  35.211750  126.841568       39
214           7  35.211194  126.842207       39
215         414  35.211194  126.842207       39
216         566  35.211194  126.842207       39
217        1702  35.210275  126.841962       39

[218 rows x 4 columns]
     Unnamed: 0        lat         lng  cluster
0          1035  35.180530  126.816932       15
1           367  35.180530  126.816932       15
2          1663  35.180530  126.816932       15
3          1192  35.212889  126.875204       20
4          1193  35.216722  126.874982       20
..          ...        ...         ...      ...
153        1376  35.199945